In [1]:
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

!nvidia-smi

cuda
Mon Nov 30 03:01:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [2]:
!pip install git+https://github.com/kernelmachine/allennlp.git@4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7
!pip install transformers==2.6.0
!pip install pytorch-transformers==1.2.0
!git clone https://github.com/allenai/dont-stop-pretraining

  Cloning https://github.com/kernelmachine/allennlp.git (to revision 4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7) to /tmp/pip-req-build-si53m7qy
  Running command git clone -q https://github.com/kernelmachine/allennlp.git /tmp/pip-req-build-si53m7qy
  Running command git checkout -q 4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 317kB 13.8MB/s 
     |████████████████████████████████| 30.9MB 170kB/s 
     |████████████████████████████████| 133kB 65.4MB/s 
     |████████████████████████████████| 5.3MB 67.7MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
     |████████████████████████████████| 266kB 60.3MB/s 
     |████████████████████████████████| 163kB 52.6MB/s 
     |████████████████████████████████| 133kB 52.0MB/s 
     |████████████████████████████████| 51kB 7.9MB/

In [3]:
%cd /content/dont-stop-pretraining
!pwd

/content/dont-stop-pretraining
/content/dont-stop-pretraining


In [5]:
import json
import pandas as pd
!curl -Lo chemtrain.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/chemprot/train.jsonl
!curl -Lo chemdev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/chemprot/dev.jsonl
!curl -Lo chemtest.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/chemprot/test.jsonl
data = []
with open("chemtrain.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))
df = pd.DataFrame(data)
df.text.to_csv("chemtrain.txt", header=False, index=False)
data = []
with open("chemdev.jsonl", 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))
df = pd.DataFrame(data)
df.text.to_csv("chemdev.txt", header=False, index=False)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1094k  100 1094k    0     0   653k      0  0:00:01  0:00:01 --:--:--  652k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  662k  100  662k    0     0   443k      0  0:00:01  0:00:01 --:--:--  443k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  937k  100  937k    0     0   618k      0  0:00:01  0:00:01 --:--:--  617k


In [6]:
!python -m scripts.run_language_modeling \
                --train_data_file chemtrain.txt \
                --line_by_line \
                --output_dir roberta_tapt_chem_01 \
                --model_type roberta-base \
                --tokenizer_name roberta-base \
                --mlm \
                --per_gpu_train_batch_size 8 \
                --gradient_accumulation_steps 32  \
                --model_name_or_path roberta-base \
                --eval_data_file chemdev.txt \
                --do_train \
                --num_train_epochs 100  \
                --learning_rate 0.0001 \
                --logging_steps 50

Streaming output truncated to the last 5000 lines.
Iteration:  45% 236/522 [00:18<00:23, 12.32it/s]
Iteration:  46% 238/522 [00:18<00:21, 13.06it/s]
Iteration:  46% 240/522 [00:18<00:22, 12.69it/s]
Iteration:  46% 242/522 [00:19<00:22, 12.55it/s]
Iteration:  47% 244/522 [00:19<00:20, 13.66it/s]
Iteration:  47% 246/522 [00:19<00:20, 13.40it/s]
Iteration:  48% 248/522 [00:19<00:24, 11.35it/s]
Iteration:  48% 250/522 [00:19<00:22, 12.07it/s]
Iteration:  48% 252/522 [00:19<00:20, 13.24it/s]
Iteration:  49% 254/522 [00:20<00:20, 13.25it/s]
Iteration:  49% 256/522 [00:20<00:20, 12.72it/s]
Iteration:  49% 258/522 [00:20<00:20, 12.68it/s]
Iteration:  50% 260/522 [00:20<00:21, 12.16it/s]
Iteration:  50% 262/522 [00:20<00:21, 11.87it/s]
Iteration:  51% 264/522 [00:20<00:22, 11.40it/s]
Iteration:  51% 266/522 [00:21<00:20, 12.60it/s]
Iteration:  51% 268/522 [00:21<00:19, 13.31it/s]
Iteration:  52% 270/522 [00:21<00:18, 13.68it/s]
Iteration:  52% 272/522 [00:21<00:21, 11.39it/s]
Iteration:  52% 27

In [7]:
!ls

ADAPTIVE_PRETRAINING.md  DATA_SELECTION.md	roberta_tapt_chem_01
chemdev.jsonl		 dont_stop_pretraining	runs
chemdev.txt		 environments		scatter.pdf
chemtest.jsonl		 environment.yml	scripts
chemtrain.jsonl		 mlm_study		search_space
chemtrain.txt		 README.md		training_config


In [8]:
from google.colab import drive

In [9]:
% cd ..

/content


In [10]:
!ls /content

dont-stop-pretraining  sample_data


In [11]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
!du -sh /content/gdrive/My\ Drive

180M	/content/gdrive/My Drive


In [16]:
!du -sh /content/dont-stop-pretraining/roberta_tapt_chem_01/

4.7G	/content/dont-stop-pretraining/roberta_tapt_chem_01/


In [18]:
!cp -r /content/dont-stop-pretraining/roberta_tapt_chem_01/ /content/gdrive/My\ Drive

In [24]:
!ls

dont-stop-pretraining  gdrive  sample_data


In [25]:
!git clone https://github.com/nsusanj/temp.git

Cloning into 'temp'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 24 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [26]:
!ls

dont-stop-pretraining  gdrive  sample_data  temp


In [28]:
!ls temp

adapticons	    Makefile   poetry.lock     README.md
docker-compose.yml  notebooks  pyproject.toml  setup.cfg


In [29]:
!python temp/adapticons/modeling/download_glue.py --data_dir 'data/glue'

Traceback (most recent call last):
  File "temp/adapticons/modeling/download_glue.py", line 151, in <module>
    sys.exit(main(sys.argv[1:]))
  File "temp/adapticons/modeling/download_glue.py", line 138, in main
    os.mkdir(args.data_dir)
FileNotFoundError: [Errno 2] No such file or directory: 'data/glue'


In [32]:
!ls temp/adapticons/modeling

download_glue.py      new_train.py
glue_training.py      run_language_modeling_with_adapter_fusion.py
jsonl_to_txt.py       run_language_modeling_with_adapters.py
load_fusion_model.py  sample_pct_txt_file.py


In [ ]:
!python temp/adapticons/modeling/new_train.py \
  --model_name_or_path /content/dont-stop-pretraining/roberta_tapt_chem_01/ \
  --task_name $TASK_NAME \
  --do_train \
  --do_eval \
  --data_dir $GLUE_DIR/$TASK_NAME \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 1e-4 \
  --num_train_epochs 10.0 \hs rsync
  --output_dir /tmp/$TASK_NAME \
  --overwrite_output_dir \
  --train_adapter \
  --adapter_config pfeiffer \
  --no_cuda

In [33]:
%cd temp

/content/temp


In [34]:
!make init_project

/bin/sh: 1: poetry: not found
/bin/sh: 1: poetry: not found
Makefile:29: *** "make sure pyenv is installed and is accessible in your path, (usually by adding to PATH variable in bash_profile, zshrc, or other locations based on your platform) See: https://github.com/pyenv/pyenv#installation for the installation insructions.".  Stop.


In [39]:
!python adapticons/modeling/download_glue.py --data_dir 'data/glue'

	Completed!
	Completed!
Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!


In [36]:
!ls

adapticons	    Makefile   poetry.lock     README.md
docker-compose.yml  notebooks  pyproject.toml  setup.cfg


In [37]:
!mkdir data

In [38]:
!ls

adapticons  docker-compose.yml	notebooks    pyproject.toml  setup.cfg
data	    Makefile		poetry.lock  README.md
